<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/Final_1_Paper_Multivariate_sliding_window_labelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install statsmodels --upgrade
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import ast
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from numpy import arange
import warnings
warnings.filterwarnings('ignore')
#from keras.utils import plot_model
#import matplotlib.pyplot as plt



df=pd.read_csv(r'/content/drive/MyDrive/PHD/2021/household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])

df.head()

#df = df[['Global_active_power','Global_reactive_power','Global_intensity']]
#Imputing NULL
df = df.replace('?', np.nan)
df.isnull().sum()

#IMputing missing value

def fill_missing(values):
    one_day = 60*24
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            if np.isnan(values[row][col]):
                values[row,col] = values[row-one_day,col]
df = df.astype('float32')
fill_missing(df.values)
df.isnull().sum()

#Downsampling to Days
daily_df = df.resample('H').sum()
daily_df.head()
#daily_df =df
ts_len = daily_df.shape[0]


#DEFINE K - User parameter - length of the LONG time series sequence.
K = 720  #taking approx 2months worth of data - for the long ts sequence

#Now convert index to column
daily_df['datetime']=daily_df.index

#VISUALISE THE TIMESERIES
fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_df['datetime'], y=daily_df['Global_reactive_power'], name='Global_reactive_power'))

fig.update_layout(showlegend=True, title='Household electricity consumption')
fig.show()

#remove index column unless required.

daily_df.drop(daily_df.columns[7], axis=1, inplace=True)
#daily_df.drop(daily_df.columns[2], axis=1, inplace=True) #dropped Voltage
#Scaling the values
whole_series = daily_df
scalers={}
for i in daily_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(whole_series[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    whole_series[i]=s_s



#Stationarity check for each time series ---- This module can be run manually for now

def augmented_dickey_fuller_statistics(time_series):
  result = adfuller(time_series.values)
  ADF_Statistic = result[0]
  p_value = result[1]
  Critical_values_1 = result[4]["1%"]
  Critical_values_5 = result[4]["5%"]
  Critical_values_10 = result[4]["10%"]

  # We take that p-value should be less than 0.05 and ADF_statistic should be less than critical value at 5% confidence Critical_value_5
  if p_value <0.05 and ADF_Statistic < Critical_values_5:
    return "stationary"
  else:
    return "non-stationary"

for i in range(0,whole_series.shape[1]):
  print('Augmented Dickey-Fuller Test Result:', whole_series.iloc[:,i].name)
  x= augmented_dickey_fuller_statistics(whole_series.iloc[:,i])
  print(x)
  #if any of the x is "non-stationary": df_difference = whole_series.diff() -- and then deal with df_difference

#We see Voltage is non-stationary - so in this case just remove it.



######## SPLIT THE TIME SERIES INTO LONG SEQUENCES OF LENGTH K

#Creating subsequences


def extract_windows_vectorized(array, large_seq_size):
    start = 0
    last_index = len(array)-1
    max_time =  last_index - large_seq_size +1  ##last index upto which sliding windoe begining can go

    sub_windows = (
        start +
        # expand_dims are used to convert a 1D array to 2D array.
        np.expand_dims(np.arange(large_seq_size), 0) +
        np.expand_dims(np.arange(max_time + 1), 0).T
    ).astype(int)

    return array[sub_windows]


large_seq_size = K
n_features = whole_series.shape[1]

Long = extract_windows_vectorized(whole_series.values,large_seq_size) #Shape: (1263, 180, 7) - 902 long sequences, each 180 long, 7 variables

maxval = Long.shape[0]
count_train = int(math.ceil(0.7*maxval))
Long_train = Long[0:count_train]
Long_test = Long[count_train:]

#First, define a function that can generate small subsequences for all the large K sequences

def generate_small_seq(series, n_past, n_future):
  #
  # n_past ==> no of past observations -- OR -- sliding window
  #
  # n_future ==> no of future observations -- prediction variable y
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)


n_future=1
predictions = list()

rmse_list = list()
min_window_list = list()
best_window_for_long_seq = list()
AIC = list()
n_fold = 6 # 5 fold plus 1 for test)
# evaluate a logistic regression model using k-fold cross-validation
from numpy import mean
from numpy import std


#for i in range(Long_train.shape[0]) :
for i in range(0,2):
  #iterate over the entire 759 long sequences of multivariate time series
  min_window_list.clear()
  for j in range(Long_train.shape[2]):
    #iterate over each variable time series
    rmse_list.clear()

    for k in range(2,(round(K)-n_fold-1)):
      Total_small_seq_X, Total_small_seq_y = generate_small_seq(Long_train[i,:,:],k, n_future)
      Total_small_seq_X_reshaped = Total_small_seq_X[:,:,j].reshape(Total_small_seq_X.shape[0],Total_small_seq_X.shape[1])
      Total_small_seq_y_reshaped = Total_small_seq_y[:,:,j].reshape(Total_small_seq_y.shape[0],Total_small_seq_y.shape[1])
      maxval2 = Total_small_seq_X_reshaped.shape[0]
      count_train2 = int(math.ceil(0.7*maxval2))
      Small_train_X = Total_small_seq_X_reshaped[0:count_train2]
      Small_train_y = Total_small_seq_y_reshaped[0:count_train2]
      Small_test_X =  Total_small_seq_X_reshaped[count_train2:]
      Small_test_y = Total_small_seq_y_reshaped[count_train2:]
      #cv = KFold(n_splits=5, random_state=1, shuffle=True)
      #regressor  = LinearRegression()
      tscv = TimeSeriesSplit(n_splits=n_fold-1)
      regressor = LassoCV(alphas=arange(0, 1, 0.01), cv=tscv, n_jobs=-1)

      #scores = cross_val_score(regressor, Total_small_seq_X_reshaped, Total_small_seq_y_reshaped, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
      try:
       regressor.fit(Small_train_X, Small_train_y) #get jth time series only
       if Small_test_X.shape[0] == 0:
        break
       y_predj = regressor.predict(Small_test_X)
       print('alpha: %f' % regressor.alpha_)
       rmse = sqrt(mean_squared_error(Small_test_y, y_predj))
       print(rmse)
       rmse_list.append(rmse)
      except:
        rmse_list.append(9999)
        print('error')



    #find minimum rmse for all sliding window, and corresponding sw size
    min_index = rmse_list.index(min(rmse_list))
    min_sw = min_index + 2
    print('i=', i,'j=', j,'SW =', min_sw)
    min_window_list.append(min_sw)

  #exited for internal loops -- now operating for each i

  cur_seq = Long_train[i,:,:]
  AIC.clear()
  model = VAR(cur_seq)

  for m in (range(min(min_window_list),max(min_window_list)+1)):
    #print(Long_train)

    results = model.fit(m)
    print('Order =', m)
    print('AIC: ', results.aic)
    print('BIC: ', results.bic)
    AIC.append(results.aic)

  minAIC_index = AIC.index(min(AIC))+min(min_window_list)

  print('Minimum lag = ', minAIC_index)
  best_window_for_long_seq.append(minAIC_index)

#OUT OF ALL LOOPS NOW
#best_window_for_long_seq now contains the best multivariate window size for each of the long sequence i
print(best_window_for_long_seq)
Window = np.array(best_window_for_long_seq)

#np.save(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences-TRAIN.npy',Long_train)
np.save(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences_WINDOW-1000.npy',Window)
#np.save(r'/content/drive/MyDrive/PHD/2021/multivariate_long_sequences-TEST.npy',Long_test)
print('saving completed')

Streaming output truncated to the last 5000 lines.
alpha: 0.020000
0.23573295271767994
alpha: 0.020000
0.23562821870472286
alpha: 0.020000
0.23529325055675235
alpha: 0.020000
0.2364006876874174
alpha: 0.020000
0.2379374133922278
alpha: 0.020000
0.23745426910972192
alpha: 0.020000
0.23913500222465994
alpha: 0.020000
0.23998476256984946
alpha: 0.020000
0.2415008424409543
alpha: 0.010000
0.227790358125396
alpha: 0.020000
0.24313739645717802
alpha: 0.020000
0.2431589608633931
alpha: 0.020000
0.24360139424953875
alpha: 0.020000
0.24446642689830542
alpha: 0.020000
0.2445711231324879
alpha: 0.020000
0.24476562608365163
alpha: 0.020000
0.2462336505869796
alpha: 0.020000
0.24647495475456765
alpha: 0.020000
0.24687569382726976
alpha: 0.020000
0.24532191780867155
alpha: 0.020000
0.2454577811526389
alpha: 0.020000
0.2457703630300016
alpha: 0.020000
0.24611680361834626
alpha: 0.020000
0.24700163840511113
alpha: 0.020000
0.24725422403528582
alpha: 0.020000
0.2475324989425075
alpha: 0.020000
0.249165

ValueError: x contains one or more constant columns. Column(s) 396, 403, 410, 417, 424, 431, 438, 445, 452, 459, 466, 473, 480, 487, 494, 501, 508, 732, 1075, 1082, 1089, 1096, 1103, 1110, 1117, 1124, 1131, 1138, 1145, 1152, 1159, 1166, 1369, 1376, 1383, 1390, 1663, 1670, 1677, 1684, 1691, 1698, 1705, 1712, 1719, 1726, 1733, 1740, 1747, 1754, 1761, 1768, 1775, 1782, 1789, 1796, 1803, 1810, 1964, 1971, 1978, 1985, 1992, 1999, 2006, 2013, 2020, 2027, 2034, 2041, 2048, 2055, 2062, 2069, 2076, 2083, 2090, 2097, 2104, 2111, 2118, 2125, 2132, 2139, 2146, 2153, 2160, 2167, 2174, 2181, 2188, 2195, 2202, 2209, 2216, 2223, 2230, 2237, 2244, 2251, 2258, 2265, 2272, 2279, 2286, 2293, 2300, 2307, 2314, 2321, 2328, 2335, 2342, 2349, 2356, 2363, 2370, 2377, 2384, 2391, 2398, 2405, 2412, 2419, 2426, 2433, 2440, 2447, 2454, 2461, 2468, 2475, 2482, 2489, 2496, 2503, 2510, 2832, 2839, 3084, 3091, 3098, 3105, 3112, 3119, 3126, 3133, 3140, 3147, 3154, 3161, 3168, 3175, 3182, 3336, 3343, 4253, 4260, 4267, 4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section